In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from collections import defaultdict
tqdm.pandas()
from ast import literal_eval

C:\Users\Markus\AppData\Local\Temp\ipykernel_34760\2954495796.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_company = pd.read_csv('data/company/Cleaned_Companies.csv')

In [3]:
df_company[
    df_company['Combined_Text'].str.contains(r'\bmanufacturing\b', case=False, na=False) &
    df_company['Combined_Text'].str.contains(r'\bhelicopter\b', case=False, na=False)
].iloc[0]['description']

'Aerotronic is a data analytics company that uses aerial and ground data to report on the health of power lines at a significant cost reduction versus existing solutions. Powerline inspections today are qualitative, done manually, and stored as hard copies. Engineers fly in a manned helicopter and take notes on the condition of the lines. Each inspection costs $2K+ per mile and takes time to collate and track. High-speed data capture using manned, unmanned, and ground-based vehicles. The data is processed using a proprietary analytics platform and accessed by clients through a web portal. Late stage discussions with 2 utility companies on pilot studies. Flew power lines in North Carolina after Hurricane Florence, and sent findings to FEMA, Duke Energy, & NCDOT. Aerotronic charges for transmission line inspections at $1000/mile (50% market rates with 80% margin). Subscription model to their portal allows for 3D time-series data access and preventative maintenance predictions using machi

In [4]:
df_company

,uuid,name,created_at,updated_at,legal_name,roles,city,status,short_description,category_list,category_groups_list,founded_on,closed_on,primary_role,description,Combined_Text,Lemmatized_Tokens,Bigrams_Tokens,founded
0,e25f4821-7be5-0e51-3962-081fe56265ac,Powerset,2007-06-12 02:33:13,2019-06-24 21:56:16,NaN,company,San Francisco,acquired,Powerset is a search engine focused on in-dept...,"Internet,Natural Language Processing,Search En...","Artificial Intelligence (AI),Data and Analytic...",2005-09-01,NaN,company,Powerset is a search engine focused on natural...,powerset is a search engine focused on in-dept...,"['powerset', 'search', 'engine', 'focus', 'dep...","['powerset', 'search', 'engine', 'focus', 'dep...",2005
1,d0647cec-acb3-9a23-bc4f-d3fad90ecb4e,STRANDS,2007-06-17 15:17:01,2021-07-26 07:43:25,NaN,company,Miami,acquired,Strands is a FinTech partner that develops dig...,"Artificial Intelligence (AI),Banking,Big Data,...","Artificial Intelligence (AI),Data and Analytic...",2004-01-01,NaN,company,Strands is a FinTech partner that develops dig...,strands is a fintech partner that develops dig...,"['strand', 'fintech', 'partner', 'develop', 'd...","['strand', 'fintech', 'partner', 'develop', 'd...",2004
2,3d02b92f-668f-1a97-5c12-5d7646ae3ffe,Aggregate Knowledge,2007-06-28 15:04:25,2021-09-20 12:24:55,"Aggregate Knowledge, Inc.",company,San Mateo,closed,Aggregate Knowledge is a media intelligence co...,"Advertising,Big Data,Predictive Analytics,Risk...","Advertising,Artificial Intelligence (AI),Data ...",2006-01-01,NaN,company,Aggregate Knowledge (AK) is the only media int...,aggregate knowledge is a media intelligence co...,"['aggregate', 'knowledge', 'medium', 'intellig...","['aggregate', 'knowledge', 'medium', 'intellig...",2006
3,84e45eb1-4a2a-19e5-8868-6def7d0fc049,SK Telecom,2007-07-20 16:32:17,2023-11-03 01:06:58,NaN,"investor,company",Seoul,ipo,SK Telecom focuses on developing innovative te...,"Artificial Intelligence (AI),Mobile Apps,Telec...","Apps,Artificial Intelligence (AI),Data and Ana...",1984-03-19,NaN,company,SK Telecom is dedicated to developing advanced...,sk telecom focuses on developing innovative te...,"['telecom', 'focus', 'develop', 'innovative', ...","['telecom', 'focus', 'develop', 'innovative', ...",1984
4,3e3e8a2f-3700-f2c9-6568-4aaae02c07c6,Qualcomm,2007-07-31 19:47:34,2024-03-08 07:51:29,"Qualcomm Technologies, Inc.","investor,company",San Diego,ipo,"Qualcomm designs, manufactures, and markets di...","Artificial Intelligence (AI),Generative AI,Sof...","Artificial Intelligence (AI),Data and Analytic...",1985-01-01,NaN,company,Qualcomm is an American fabless semiconductor ...,"qualcomm designs, manufactures, and markets di...","['qualcomm', 'design', 'manufacture', 'market'...","['qualcomm', 'design', 'manufacture', 'market'...",1985
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54832,71e1d6f5-aa93-425d-a348-bc6cde7f8cb2,Keychain,2024-06-17 15:18:42,2024-06-17 15:18:42,Keychain,company,New York,operating,AI Platform,"Artificial Intelligence (AI),Food and Beverage...","Artificial Intelligence (AI),Data and Analytic...",2023-11-01,NaN,company,Keychain is an innovative startup transforming...,ai platform. keychain is an innovative startup...,"['ai', 'platform', 'keychain', 'innovative', '...","['ai', 'platform', 'keychain', 'innovative', '...",2023
54833,f98de4a0-3cfa-482d-bba0-a210cd9c7e76,Menuz,2024-06-17 15:45:42,2024-06-17 15:53:29,NaN,NaN,San Francisco,operating,AI menu personalized recommendation and QR cod...,"Artificial Intelligence (AI),Food and Beverage","Artificial Intelligence (AI),Data and Analytic...",2024-01-01,NaN,company,NaN,ai menu personalized recommendation and qr cod...,"['ai', 'menu', 'personalize', 'recommendation'...","['ai', 'menu', 'personalize', 'recommendation'...",2024
54834,a7d94ffe-e989-4d67-9532-efc842f3a668,Escalate Digital,2024-06-17 15:55:55,2024-06-17 15:55:55,Escalate Digital,company,NaN,operating,we specialize in delivering impact

In [5]:
#filter to time frame and only companies
df_company = df_company[(df_company['founded']>=1990) & (df_company['founded']<=2023)]
df_company = df_company[df_company['primary_role']=='company']

In [8]:
df_edges = pd.read_csv('data/edges/tagged_edges.csv') #any edge list is sufficient
df_edges = df_edges.drop_duplicates(subset=['source', 'target'], keep='first')

In [9]:
df_company['Bigrams_Tokens'] = df_company['Bigrams_Tokens'].progress_apply(literal_eval)

  0%|          | 0/51992 [00:00<?, ?it/s]

In [12]:
df_company = df_company.sort_values(by='founded')

In [13]:
df_company['Bigrams_Tokens'] = df_company['Bigrams_Tokens'].apply(set)

In [16]:
#semi slow
def find_first_occurrence(source, target):
    mask = df_company['Bigrams_Tokens'].apply(lambda x: source in x and target in x)
    
    filtered_df = df_company[mask]
    
    if not filtered_df.empty:
        first_row = filtered_df.iloc[0]
        return first_row.name, first_row['founded']
    return 0, 0

In [17]:
df_edges[['index','company_year']] = df_edges.progress_apply(
    lambda row: find_first_occurrence(row['source'], row['target']),
    axis=1,
    result_type='expand'
)

  0%|          | 0/70671 [00:00<?, ?it/s]

In [22]:
def determine_color(row):
    if row['company_year'] == 0:
        return 1 #exclusive to patents
    elif row['company_year'] < row['year']:
        return 2 #in patents before company
    elif row['company_year'] > row['year']:
        return 3 #in company before patents
    else:
        return 4 #same year in both

In [23]:
df_edges['company_color'] = df_edges.apply(determine_color, axis=1)

In [24]:
exclusive_patents = len(df_edges[df_edges['company_color'] == 1])
patents_before_company = len(df_edges[df_edges['company_color'] == 2])
company_before_patents = len(df_edges[df_edges['company_color'] == 3])
same_year_both = len(df_edges[df_edges['company_color'] == 4])

total = exclusive_patents + patents_before_company + company_before_patents + same_year_both

print("exclusive to patents:", exclusive_patents)
print("in patents before company:", patents_before_company)
print("in company before patents:", company_before_patents)
print("same year in both:", same_year_both)
print("total:", total)

exclusive to patents: 44808
in patents before company: 8784
in company before patents: 16211
same year in both: 868
total: 70671


In [25]:
df_edges.to_csv('data/edges/edges_with_company_colors.csv',index=False)